In [ ]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import torch
import torchmetrics
from pathlib import Path
from src.modules.lcm_module import LCMModule 
from src.utils.misc_utils import count_params, run_evaluation_experiments

import warnings
warnings.filterwarnings("ignore")

roc = torchmetrics.classification.BinaryROC()
auroc = torchmetrics.classification.BinaryAUROC()

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# for reproducibility
np.random.seed(42)
torch.manual_seed(42)

### Training Ablations

In [ ]:
""" Paths - adjust as needed """
par_dir = Path.cwd().parent
models_path = Path("/media/nikolas/KINGSTON/LCM/logs/ablations_model")
cp_path = Path(par_dir / "src/benchmarks/CausalPretraining/pretrained_weights")
out_path = Path(par_dir / "outputs")

try:
    out_path.mkdir(parents=True, exist_ok=False)
    print(f"Created: {out_path}")
except FileExistsError:
    print("Output path already exists.")

models = {
    "Informer_5V_3L": LCMModule.load_from_checkpoint(Path(models_path / "Informer_5V_3L.ckpt")),
    "LCM_5V_3L_CI": LCMModule.load_from_checkpoint(Path(models_path / "LCM_5V_3L_CI.ckpt")),
    "LCM_5V_3L_CI_CR_1.0": LCMModule.load_from_checkpoint(Path(models_path / "LCM_5V_3L_CI_CR_1.0.ckpt")),
    "LCM_5V_3L_CI_CR_0.25": LCMModule.load_from_checkpoint(Path(models_path / "LCM_5V_3L_CI_CR_0.25.ckpt")),
    "LCM_5V_3L_CI_CR_0.5": LCMModule.load_from_checkpoint(Path(models_path / "LCM_5V_3L_CI_CR_0.5.ckpt")),
    "LCM_5V_3L_CI_CR_0.75": LCMModule.load_from_checkpoint(Path(models_path / "LCM_5V_3L_CI_CR_0.75.ckpt")),
}

for model_name, model in models.items():
    print(f"\n___{model_name}___")
    print(count_params(model, pretty=False))

#### Test Synthetic

In [ ]:
run_evaluation_experiments(models=models, cpd_path=Path("/media/nikolas/KINGSTON/Datasets/synthetic_data/S_/S_joint/test"),
                           out_dir=out_path, N_RUNS=5)

### Synthetic vs Simulated

In [ ]:
# Paths - adjust as needed
par_dir = Path.cwd().parent

models_path = Path(par_dir / "/media/nikolas/KINGSTON/LCM/logs/ablations_sim") 
out_path = Path(par_dir / "outputs")

try:
    out_path.mkdir(parents=True, exist_ok=False)
    print(f"Created: {out_path}")
except FileExistsError:
    print(f"{out_path} already exists.")

models = {
    "LCM_100syn_0sim": LCMModule.load_from_checkpoint(Path(models_path / "LCM_100syn_0sim.ckpt")),
    "LCM_80syn_20sim": LCMModule.load_from_checkpoint(Path(models_path / "LCM_80syn_20sim.ckpt")),
    "LCM_50syn_50sim": LCMModule.load_from_checkpoint(Path(models_path / "LCM_50syn_50sim.ckpt")),
    "LCM_20syn_80sim": LCMModule.load_from_checkpoint(Path(models_path / "LCM_20syn_80sim.ckpt")),
}

for model_name, model in models.items():
    print(f"\n___{model_name}___")
    print(count_params(model, pretty=False))

#### Test fMRI_5

In [ ]:
fmri_path = Path(par_dir / "data/fMRI_5")

run_evaluation_experiments(models=models, cpd_path=fmri_path, out_dir=out_path, fmri_data=True)

#### Test fMRI_10

In [ ]:
""" Path """
fmri_path = Path(par_dir / "data/fMRI")

run_evaluation_experiments(models=models, cpd_path=fmri_path, out_dir=out_path, fmri_data=True)

#### Test Kuramoto_5V_1L

In [ ]:
run_evaluation_experiments(models=models, cpd_path=Path("/media/nikolas/KINGSTON/Datasets/kuramoto_5V_1L/test"),
                           out_dir=out_path, sharded_data=False, kuramoto_data=True, MAX_VAR=5)

#### Test Kuramoto_10V_1L

In [ ]:
run_evaluation_experiments(models=models, cpd_path=Path("/media/nikolas/KINGSTON/Datasets/kuramoto_10V_1L/test"),
                           out_dir=out_path, sharded_data=False)

#### Test air_quality_mini

In [ ]:
run_evaluation_experiments(
    models=models,
    cpd_path=Path("/media/nikolas/KINGSTON/Datasets/air_quality_mini"),
    out_dir=out_path,
    sharded_data=False
)